# TouchTerrain jupyter notebook - starter edition, running on binder(!)
Chris Harding, Aug. 15 , 2022 (<charding@iastate.edu>)

- This notebook is meant for users who are new to jupyter notebooks and Python. It walks through the process of using TouchTerrain with many more small steps at a slow pace, makes less assumptions and gives more explanations - but will also hide some advanced details.
- It assumes that you are using it via binder i.e. `https://mybinder.org/v2/gh/ChHarding/TouchTerrain_for_CAGEO/HEAD` as there are a some special steps to take.
- This notebook is a companion to the more generic TouchTerrain_standalone_jupyter_notebook.ipynb. Once you've worked through the starter notebook and are more familiar with TouchTerrain, you may want to graduate to it as it can be more efficient and faster to work with.
- Before we start, you may want to rename this notebook (e.g. into TouchTerrain_starter_chris.ipynb) to make sure it doesn't get overwritten by accident. 

### Preparing to use Google Earth Engine online DEM rasters 

- *You don't need this if you only want to use terrain data from locally stored raster files (geotiffs) instead of online DEMs!*
- TouchTerrain can use DEM data from Google Earth Engine (given the corners of the area), but you need to first set up an earth engine account and create Google cloud project. Note that this includes setting up a charge account, however, you will typically not be charged. 

!!! MORE DETAILS TO COME !!!


- Below is a python *code* cell.
- Run the cell  by clicking inside it and pressing Shift + Enter. Or you can hit the green > Run button at the top left.
- You wont see any output but you'll see that the braces in `In [ ]` will show a sequential number, which means it's been run.
- You will have to run this cell every time to start with a "fresh"notebook (either at the very beginning or after you've rebooted the kernel)


In [1]:
import ee # import module for earthengine-api 

### Authentication with Earth Engine
- the next cell will initiate an authorization with the earth-engine . It's quite painless and uses a browser
- remove the leading `#` (uncomment the line) and run it. Make sure there's no space in front of ee!
- This text should appear and you should be redirected to a webpage:
```
To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does  not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=5172225062...

```
- The web page will have you select a Google account for use with Earth Engine and give you a code (copy it)
- Paste in your code here: `Enter verification code: <your code>` and hit Enter, you should get `Successfully saved authorization token.`
- Important: this needs to be done only once when using Touchterrain. Once done, you should put the `#` back in front of ee.Authenticate() (so the line is `#ee.Authenticate()` again), so you're not running it again by accident. But, if you ever remove or update the image, you'll have to authenticate again.

In [ ]:
#ee.Authenticate() # authenticate your earth engine to consume DEM data from Google

### Installing additional packages: gdal, geemap and k3d
- There are a few additional packages that the notebook version of TouchTerrain needs (i.e. which are not covered by setup.py)
- geemap and k3d will install easily with pip
- For gdal, we'll use conda, which I cannot get to install with pip


#### Installing gdal
- run the cell below to install gdal (! means that it will be run inside a OC shell instead of as Python code
- Note: On binder we are running a virtual environment called notebook, in which gdal will be install
- This includes downloading a bunch of dependencies, will generate a lot of messages (that you can sefely ignore) and may take some time

In [9]:
!conda install -y -c conda-forge gdal

Solving environment: done

# All requested packages already installed.



#### Installing k3d and geemap
- run the next line to install k3d and geemap using pip 
- Again, this may take some time and will show you lots of messages

In [10]:
!pip install k3d geemap 

### Importing python packages
- Run the next cell, it'll import the required packages into Python, including Touchterrain
- The earth engine package you imported and authorized earlier, will be initialized here. You should see `EE init() worked with .config/earthengine/credentials`


In [11]:
# import packages
import os.path
from glob import glob
import k3d
from random import randint
from shutil import rmtree
import zipfile
from geojson import Polygon
import geemap
from touchterrain.common import TouchTerrainEarthEngine as TouchTerrain
from touchterrain.common.TouchTerrainGPX import *
args = TouchTerrain.initial_args # default args


## Define the values for processing settings
- We now need to define the processing settings for the 3D model we want to create. Unlike the web app, they need to be set via Python variables.
- The variable `args` will hold all processing settings. The __name__ of a setting  will be between [] and inside double quotes, the setting's __value__ will be to the right of the = . The value can be a number (120 or 0.4), a string ("STLb") or the special value `None`.
- Example: `args["tilewidth"] = 120`     will define the value of the setting named `tilewidth` to be 120

### Comments
- In Python, anything to the right of a `#` will be ignored.
- You will often need comment out or uncomment a line of Python code.
- A line starting with a `#` (no spaces before it) is said to be commented out, it will be entirely ignored. This a useful to termporarily deactivate the line and re-activate it again later.
- To uncomment a line means to remove the leading `#`, again leaving no spaces before the first actual code letter. This makes the line valid code again.

- Example of commenting out a line of code and uncommenting it properly:
```
print(123)
#print(123)
print(123)
```
- Examples of bad commenting with a leading space. I'm using an underscore here for better visibility.
```
_#print(123)
_print(123)
```

### Use a locally stored geotiff DEM file instead of an online DEM
__If you don't plan to use a local geotiff and instead want to use an online DEM, skip this cell and go to the next__
- You may want create a terrain model from a geotiff DEM file that you've downloaded or created via a GIS. As TouchTerrain needs to import this file during processing, you first need to copy (upload) it via the jupyter file manager (tree).
- Go to the same folder as your .ipynb notebook file and hit _Upload_ in the upper right corner. This will copy a geotiff from your native OS into the container.
- Alternatively, you could make a separate data folder first, again in the  same folder as your .ipynb notebook file, something like mydata. Navigate into this folder and upload your geotiff(s) there
- The folder called __stuff__ contains a very small example geotiff file called __pyramid.tif__. From the folder you notebook is in, the path to this file is therefore __stuff/pyramid.tif__ (Unix uses a /, not a \ like Windows, for folders!)
- The cell below set the value of __importedDEM__ to this path (`stuff/pyramid.tif`). To import your file, replace this with the name of your geotiff (if you uploaded it the notebook's folder) e.g. myDEM.tif. If you're using your own data folder prepend the folder's name in frony of the filename, e.g. mydata/myDEM.tif. Note that this must not start with a / !
- if you do NOT want to use a local geotiff and want to use online DEM rasters instead, set the value to None. Comment/Uncomment the appropriate choice below and hit Shift-Enter
- if you are using a local geotiff, skip _Select DEM source_ and _Select print area on geemap_ as the geotiff is your DEM source and  implicitly defines the print area. Go straight to _Setting the tile width_

In [8]:
# Comment out one of following two lines:
#args["importedDEM"] =  "stuff/pyramid.tif" # location of local geotiff to use
args["importedDEM"] = None  # use online DEM rasters instead

# convert into an absolute path for later
if args["importedDEM"] != None:
    args["importedDEM"]= os.path.abspath(args["importedDEM"]) 
    print("importedDEM", args["importedDEM"])

### Select the name of the DEM source
- Before you can select the actual print area you need to select which online DEM source will be used. The setting is called `DEM_name`.
- There are several options but for starters you only need to decide among two options:
    - If your area is in the lower 48 US states, use `USGS/NED`
    - Anywhere else on the globe use: `JAXA/ALOS/AW3D30/V2_2`
- Again, comment/uncomment the appropriate line in the cell below and hit Shift-Enter

In [6]:
# Comment out one of the following two lines
args["DEM_name"] = "USGS/3DEP/10m"  # area within the lower 48 (US)
#args["DEM_name"] = "JAXA/ALOS/AW3D30/V2_2"  # area is outside the US (worldwide)

### Import GPX path lines
__This is more of an expert option, if you are just starting, just skip this and go to the next cell__
- You can drape one or more gpx (path) files over your terrain
- Similar to importedDEM, the file names for the GPX file(s) are stored in the importedGPX setting.
- However, as you can import more than one GPX file, you need to put the path name into a list, i.e. inside brackets, even if you just have one GPX file. Separate multiple GPX files with a comma:  
    - `args["importedGPX"] = ["justonegpxfile.gpx"]`  
    - `args["importedGPX"] = ["file1.gpx", "file2.gpx"]`
- `gpxPathHeight` (in meters) defines how much a path is elevated above the terrain. Use a negative number to create a trench.
- `gpxPixelsBetweenPoints` (in meters) lets you reduce the number of point in your path to place a point only every X meters. This can help to simplify complex patterns.
- `gpxPathThickness` (meters) controls the thickness of the path, which makes it more pronouced when printed
- There are several GPX example files in the stuff folder. To use them, uncomment the first 4 lines in the cell below and comment out the last line.
- If you're not using GPX files, just leave the cell as is and run it. Setting importedGPX to None means that no GPX files are imported.


In [7]:
args["gpxPathHeight"] = 5
args["gpxPixelsBetweenPoints"] = 20
args["gpxPathThickness"] = 2

# Comment out one of the following two lines
#args["importedGPX"] = ["stuff/gpx-test/DLRTnML.gpx", "stuff/gpx-test/DonnerToFrog.gpx", "stuff/gpx-test/CinTwistToFrog.gpx"] # list of GPX files
args["importedGPX"] = None  # Do not use any GPX path files


## Select your print area
-There are several ways to define the area you want to print. Use only one and skip the cells for the other two options. The go to _Setting th tile width_
- A)  Enter corner coordinates of a box into a code cell
- B)  Use your mouse to digitize a box, circle or polygon on a geemap
- C)  Load a kml file (typically created via Google Earth) that contains a single polygon


### A) Select print area via lat/long coordinates of its corners (optional)
- If you'd rather define your area through actual coordinates, change the default values in the cell below accordingly. 
- Note that you *must* give it the lat/long of the bottom left (South-West) corner and lat/long of the top right (North-East) corner!
- After you've run the cell, jump over the geemap selection and the kml file outline (which would override your settings) and jump straight to `Setting the tile width`

In [ ]:
# Bottom left corner
args["bllat"] = 39.322
args["bllon"] = -120.374

# Top right corner coordinates
args["trlat"] = 39.457
args["trlon"] = -120.2



### B) Select print area interactively via geemap
- If you didn't use a local geotiff or set your the print area's corner coordinates, run the next cell to show an interactive map, from which you will select your print area. 
- Valid areas are covered by a grey hillshade layer in the background. If your desired area is not covered, go back and select the worldwide DEM source, and re-run the next cell.
- If you're importing GPX files, they will show up on the map as cyan lines.
- To find the desired print area, use left mouse drag to pan and mouse wheel (or +- buttons) to zoom in/out.
- You can also use the search buttons (globe on top left or spyglass) which will place a marker at the result's location.

In [ ]:
# various things I tried to solve the model not found error ....

In [21]:
!pip uninstall -y ipywidgets ipyleaflet ipyevents

Found existing installation: ipywidgets 7.5.1
Uninstalling ipywidgets-7.5.1:
  Successfully uninstalled ipywidgets-7.5.1
Found existing installation: ipyleaflet 0.17.0
Uninstalling ipyleaflet-0.17.0:
  Successfully uninstalled ipyleaflet-0.17.0
Found existing installation: ipyevents 2.0.1
Uninstalling ipyevents-2.0.1:
  Successfully uninstalled ipyevents-2.0.1


In [22]:
!pip install ipywidgets==7.5.1

  Using cached ipywidgets-7.5.1-py2.py3-none-any.whl (121 kB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
geemap 0.16.4 requires ipyevents, which is not installed.
geemap 0.16.4 requires ipyleaflet>=0.17.0, which is not installed.


In [ ]:
!pip install qgrid --upgrade 

In [29]:
!jupyter labextension install @jupyter-widgets/jupyterlab-manager

Building jupyterlab assets (production, minimized)
An error occurred.
RuntimeError: JupyterLab failed to build
See the log file for details:  /tmp/jupyterlab-debug-k1n4h_ya.log


In [23]:
# Create an interactive map and center on default area
center_lat = (args["trlat"] + args["bllat"]) / 2
center_lon = (args["trlon"] + args["bllon"]) / 2
Map = geemap.Map(center=(center_lat, center_lon), zoom=7) 

# make a hillshade layer and add it to map
dem = ee.Image(args["DEM_name"]) # DEM source
hs = ee.Terrain.hillshade(dem, 315, 35) # sun azimuth and angle 
vis_params = {'min': 0,'max': 255,} # greyscale color ramp
Map.addLayer(hs, vis_params, 'hillshade', shown=True, opacity=0.5) # semi transparent overlay

# if GPX files were used, add them to the map
if args["importedGPX"] != None and len(args["importedGPX"]) > 0: 
        gpx = ee.Feature(convert_to_GeoJSON(args["importedGPX"]))
        Map.addLayer(gpx, {"color":'00FFFF', "strokeWidth":"1"}, "GPX line", opacity=0.9)

Map # makes the interactive map show up as output of this cell

Map(center=[39.389844274126574, -120.28760044432504], controls=(WidgetControl(options=['position', 'transparen…

- Once you found a good general area, hit a Draw button in the center left to digitize the exact outline of your terrain model. 
- Your options are: Draw a Polygon or Draw a Rectangle or Draw a Circle. Do NOT use Draw a Marker or a Polyline!
- Draw a Rectangle is the simplest method.
- Note:I do not know how to edit or delete outlines. If you change you mind you can simply draw another outline on top of previous outline, which will then be used. If this gets too cluttered, you can always re-run the cell above.
- Once you're happy with your outline, run the cell below. It will store the outline and use it to create your terrain model. If you change your mind, simply draw another outline and re-run the cell below, it will always take the most recent outline.

In [ ]:
# make and store a GeoJSON polygon from (last) digitized outline
polyft = Map.draw_last_feature # get last outline
args["polygon"] = Polygon(polyft.getInfo()['geometry']['coordinates'])

### C) Using a KML file to define the outline of the area
- Instead of digitizing an outline in the geemap, you could instead use a polygon stored in a kml file. You can digitize a polygon in Google Earth and store it as a kml file (NOT a kmz file!). 
- Again, place the kml file in the same folder as the notebook (or in a subfolder) and set `poly_file` to its location.
- The stuff folder contains an example kml file

In [9]:
# Comment out either this line:
args["poly_file"] = None # Don't use a kml file

# or both of these lines:
args["poly_file"] = "stuff/polygon_example.kml" # location of kml file to use
#args["polygon"] = None  # ensure that any gee polygon you might have digitized is not used

### Setting the tile width
- (if you used a local geotiff file as DEM, resume here)
- the tilewidth setting defines how large the selected print area will be after it's been printed. Units are in mm.
- By width, we mean the extent of you model in the East-West direction. The height will be automatically calculated.
- Change the value in the cell below to your liking and hit Shift-Enter
- Although this can be fixed later, you should set the width to be less than the widest dimension of your printer's build plate.

In [10]:
args["tilewidth"] = 120 # in mm

### Setting the number of tiles and their and setup  
- It's possible to divide the print are into multiple tiles. This will results in several STK files instead of just one.
- This is useful if you want to print several (smaller) tiles that each fit on you buildplate and later glue them together into one large model.
- If you just want a single STL file, set both, ntilesx and ntilesy, to 1
- For multiple tiles, decide how many tiles you want along East-West (ntilesx) and how many along North-South (ntilesy)

In [11]:
args["ntilesx"] = 1 # number of tiles in x  
args["ntilesy"] = 1 # number of tiles in y    

### Setting the Base thickness (in mm)
- The basethick setting determines how much material is put beneath the thinnest part to the terrain print 
- Use a value of at least 0.5 mm, more if you want a beefier base

In [12]:
args["basethick"] = 0.6 # in mm

### Setting the print resolution
- Set this value to the diameter of you nozzle size or slightly below. 
- Tinker with this only if you know what you're doing. This setting defines how much of the fine details the STL file will contain. The detail you're realistically able to print is limited by your nozzle size. If you set setting much lower than your nozzle size, the STL file will be needlessly large, the slicer will take longer but your print will come out the same as if you had used approximately your nozzle size.
- In almost all cases, 0.4 mm will be fine for 3D printers.
- If you are using a local geofiff file, setting this to -1 will use the native resolution of the geotiff. This is typically overkill b/c of the nozzle limitation and may lead to huge models that may be too much for you system to handle!

In [13]:
args["printres"] = 0.4 # in mm

### Setting the z-scale (elevation exaggeration factor)
- To print the terrain in its "true" form (without any vertical scaling applied), set zscale to 1.0. This works well for terrain with great elevation differences, such as the Grand Canyon or Mt. Fuji
- Most other terrain usual benefits from a z-scale larger than 1. For gentle hills, use 1.5 to 2.5. For cities use 2.5 to 6. For river deltas and other low relief areas use up to 10.
- The height of your printed terrain should be at least 20 - 30 mm high to show good detail. You can check the processing log for `top min/max`, max should be least 20 - 30 mm. If this is too low, use a higher zscale and process again.
- Alternatively, you can set this to a __negative number__ which will be interpreted as the desired height of the tallest terrain point in mm. Examples:  -12.7 means "make 1/2 inch tall", -20 means "make 20 mm tall", etc.

In [14]:
args["zscale"] = 1.5 # elevation scale factor

### Set the name of your terrain model zipfile
- Once a STL file of your terrain model has been created, it will be zipped into a zipfile and copied into the tmp folder. The zipfile also contains a log file and a geotiff.
- It's often a good idea to give the zipfile a name that reflects the terrain, e.g. grand_canyon.zipfile
- Set zip_file_name to the name of your zipfile. Dot not add the .zip extension, this will be done automatically
- To 3D print you model, download the zip file from the container to your native OS

In [15]:
args["zip_file_name"] = "myterrain" # terrain model will be inside tmp/myterrain.zip

### Other settings
- There are several more expert settings which we will skip for now.
- If you're interested, look here: https://github.com/ChHarding/TouchTerrain_for_CAGEO#processing-parameters
- If you're interested, run the cell below to list all setting names and their values


In [16]:
for name in args:
    print(name, args[name])

DEM_name USGS/3DEP/10m
bllat 39.32205105794382
bllon -120.37497608519418
trlat 39.45763749030933
trlon -120.2002248034559
importedDEM None
printres 0.4
ntilesx 1
ntilesy 1
tilewidth 120
basethick 0.6
zscale 1.5
fileformat STLb
tile_centered False
zip_file_name myterrain
CPU_cores_to_use 0
max_cells_for_memory_only 25000000
no_bottom False
bottom_image None
ignore_leq None
lower_leq None
unprojected False
only None
importedGPX None
smooth_borders True
offset_masks None
fill_holes None
poly_file stuff/polygon_example.kml
gpxPathHeight 5
gpxPixelsBetweenPoints 20
gpxPathThickness 2


## Generating the STL model file
- Running the next cell will convert the DEM (either from a local geotiff file or from a Google Earth online DEM source) into STL file and put it inside a zipfile in the tmp folder. You will also get a log file and the geotiff that was processed.
- This may take some time!
- During processing you'll see a star indicator (In[*]) and a lot of log messages. Those messages will also be in the logfile inside the zip, and may be useful later.

In [17]:
# create zipfile under tmp with the STL file of the terrain model
totalsize, full_zip_file_name = TouchTerrain.get_zipped_tiles(**args) 
print("Created zip file", full_zip_file_name,  "%.2f" % totalsize, "Mb")

INFO:root:Read file KML polygon with 28 points from stuff/polygon_example.kml
INFO:root:Log for creating 3D model tile(s) for  10m_-122.19_46.20 
 
INFO:root:DEM_name = USGS/3DEP/10m 
INFO:root:trlat = 46.2432609613901 
INFO:root:trlon = -122.15305471195137 
INFO:root:bllat = 46.15378648649877 
INFO:root:bllon = -122.22804523936263 
INFO:root:printres = 0.4 
INFO:root:ntilesx = 1 
INFO:root:ntilesy = 1 
INFO:root:tilewidth = 120 
INFO:root:basethick = 0 
INFO:root:zscale = 1.5 
INFO:root:fileformat = STLb 
INFO:root:no_bottom = False 
INFO:root:unprojected = False 
INFO:root:no_normals = True 
INFO:root:
process started: 20:49:29.782200 
INFO:root:
Region (lat/lon):
   46.2432609613901 -122.15305471195137 (top right)
   46.15378648649877 -122.22804523936263 (bottom left) 
INFO:root:center at [-122.190549975657, 46.19852372394443]  UTM 10 N ,  EPSG:32610 
INFO:root:lon/lat size in degrees: [0.07499052741125922, 0.08947447489133253] 


Log for creating 3D model tile(s) for  10m_-122.19_46.20 
 
DEM_name = USGS/3DEP/10m 
trlat = 46.2432609613901 
trlon = -122.15305471195137 
bllat = 46.15378648649877 
bllon = -122.22804523936263 
printres = 0.4 
ntilesx = 1 
ntilesy = 1 
tilewidth = 120 
basethick = 0 
zscale = 1.5 
fileformat = STLb 
no_bottom = False 
unprojected = False 
no_normals = True 

process started: 20:49:29.782200 

Region (lat/lon):
   46.2432609613901 -122.15305471195137 (top right)
   46.15378648649877 -122.22804523936263 (bottom left) 
center at [-122.190549975657, 46.19852372394443]  UTM 10 N ,  EPSG:32610 
lon/lat size in degrees: [0.07499052741125922, 0.08947447489133253] 
requesting 19.294004065718443 m resolution from EarthEngine


INFO:root:Earth Engine raster: USGS/3DEP/10m 


Earth Engine raster: USGS/3DEP/10m 


INFO:root:URL for geotiff is:  https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/0033fd3be589beb36d9c348b940855ff-7edc1153fcb155df0965baf9a17cdfaa:getPixels 


URL for geotiff is:  https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/0033fd3be589beb36d9c348b940855ff-7edc1153fcb155df0965baf9a17cdfaa:getPixels 


INFO:root: geotiff size: 0.3565378189086914 Mb 
INFO:root: cell size 19.294004065718443 m, upper left corner (x/y):  559506.8239017691 5121439.027212435 
INFO:root:omitting cells with elevation < -16384 
INFO:root:full (untiled) raster (height,width)  (520, 306) float64 elev. min/max: 1092.9068603515625 2546.7685546875 
INFO:root:cell size: 19.294004065718443 m  
INFO:root:adjusted print res from the requested 0.4 mm to 0.39215686274509803 mm to ensure correct model dimensions 
INFO:root:total model size in mm: 120 x 203.92156862745097 
INFO:root:map scale is 1 : 49199.71036758203 
INFO:root:Cells per tile (x/y) 306 x 520 
INFO:root:using single-core only 


 geotiff size: 0.3565378189086914 Mb 
 cell size 19.294004065718443 m, upper left corner (x/y):  559506.8239017691 5121439.027212435 
omitting cells with elevation < -16384 
full (untiled) raster (height,width)  (520, 306) float64 elev. min/max: 1092.9068603515625 2546.7685546875 
cell size: 19.294004065718443 m  
adjusted print res from the requested 0.4 mm to 0.39215686274509803 mm to ensure correct model dimensions 
total model size in mm: 120 x 203.92156862745097 
map scale is 1 : 49199.71036758203 
elev min/max : 1092.91 to 2546.77
Cells per tile (x/y) 306 x 520 
using single-core only 
processing tile: 1 1
top min/max: 0.6 44.92531259250752


creating internal triangle data structure for <_MainProcess(MainProcess, started)>
10 % <_MainProcess(MainProcess, started)>
20 % <_MainProcess(MainProcess, started)>
30 % <_MainProcess(MainProcess, started)>
40 % <_MainProcess(MainProcess, started)>
50 % <_MainProcess(MainProcess, started)>
60 % <_MainProcess(MainProcess, started)>
70 % <_MainProcess(MainProcess, started)>
80 % <_MainProcess(MainProcess, started)>
90 % <_MainProcess(MainProcess, started)>
100% <_MainProcess(MainProcess, started)> 

assembling binary stl from 441976 triangles  None
10 %, 20 %, 30 %, 40 %, 50 %, 60 %, 70 %, 80 %, 90 %, 100 %, 

tile 1 1 STLb 21.075138092041016 Mb 
INFO:root:
1 x 1 tiles, tile size 120.00 x 203.92 mm
 
INFO:root:tile 1 1 : height:  0.6 - 44.92531259250752 mm , file size: 21 Mb 
INFO:root:
total size for all tiles: 21 Mb 
INFO:root:added full geotiff as 10m_-122.19_46.20.tif 
INFO:root:
processing finished: 20:49:48.089987 



1 x 1 tiles, tile size 120.00 x 203.92 mm
 
tile 1 1 : height:  0.6 - 44.92531259250752 mm , file size: 21 Mb 

total size for all tiles: 21 Mb 
zip finished: 20:49:48.087314
added full geotiff as 10m_-122.19_46.20.tif 

processing finished: 20:49:48.089987 
Created zip file tmp/myterrain.zip 21.43 Mb


## Preview the model
- run the cell below to get a 3D Preview of the model. If you have multiple tiles they will have different colors.

In [ ]:
!pip install --upgrade qgrid>=1.1.1

In [20]:
!jupyter labextension install @8080labs/qgrid

Building jupyterlab assets (production, minimized)
An error occurred.
RuntimeError: JupyterLab failed to build
See the log file for details:  /tmp/jupyterlab-debug-76qn8h3j.log


In [21]:
!jupyter labextension list

JupyterLab v3.3.3
/srv/conda/envs/notebook/share/jupyter/labextensions
        jupyterlab_pygments v0.2.2 enabled OK (python, jupyterlab_pygments)
        jupyter-offlinenotebook v0.2.2 enabled OK
        ipytree v0.2.1 enabled OK
        k3d v2.14.5 enabled OK (python, k3d)
        jupyterlab-plotly v5.10.0 enabled OK
        bqplot v0.5.34 enabled OK (python, bqplot)
        ipyevents v2.0.1 enabled OK
        jupyter-leaflet v0.17.0 enabled OK
        @jupyter-widgets/jupyterlab-manager v3.1.1 enabled OK (python, jupyterlab_widgets)
        @jupyter-server/resource-usage v0.6.1 enabled OK (python, jupyter-resource-usage)

Other labextensions (built into JupyterLab)
   app dir: /srv/conda/envs/notebook/share/jupyter/lab
        @8080labs/qgrid v1.1.1 enabled OK


Build recommended, please run `jupyter lab build`:
    @8080labs/qgrid needs to be included in build


In [26]:
!jupyter lab build  --dev-build=False --minimize=False

[LabBuildApp] JupyterLab 3.3.3
[LabBuildApp] Building in /srv/conda/envs/notebook/share/jupyter/lab
[LabBuildApp] Building jupyterlab assets (production, not minimized)
Build failed.
Troubleshooting: If the build failed due to an out-of-memory error, you
may be able to fix it by disabling the `dev_build` and/or `minimize` options.

If you are building via the `jupyter lab build` command, you can disable
these options like so:

jupyter lab build --dev-build=False --minimize=False

You can also disable these options for all JupyterLab builds by adding these
lines to a Jupyter config file named `jupyter_config.py`:

c.LabBuildApp.minimize = False
c.LabBuildApp.dev_build = False

If you don't already have a `jupyter_config.py` file, you can create one by
adding a blank file of that name to any of the Jupyter config directories.
The config directories can be listed by running:

jupyter --paths

Explanation:

- `dev-build`: This option controls whether a `dev` or a more streamlined
`producti

In [24]:
# unzip zif file into a folder a
folder, file = os.path.splitext(full_zip_file_name) 
zip_ref = zipfile.ZipFile(full_zip_file_name, 'r')
zip_ref.extractall(folder)
zip_ref.close()

# get all stl files (tiles) in that folder
mesh_files = glob(folder + os.sep + "*.STL")

# Create 3D plot
plot = k3d.plot()

# Add all tiles with a random color
for m in mesh_files:
    col = (randint(0,255) << 16) + (randint(0,255) << 8) + randint(0,255) # random rgb color as hex
    buf = open(m, 'rb').read()
    plot += k3d.stl(buf, color=col)
plot.display()

# remove folder
rmtree(folder)

NameError: name 'full_zip_file_name' is not defined

## Final thoughts
- You can now download your zipfile from the tmp folder and unzip it to 3D print the STL file.
- In the jupyter file manager (tree), navigate to the folder that contains the file (tmp) and select the zip file via the checkbox on the left. A Download button will appear at the top, click it to download the zip file to your native operating system (Win10 or Mac).
- If you want to upload files, use the Upload button in the upper right.

### How to quit and resume working on this notebook
- As you worked through the notebook, you changed the content of its cells and also created output cells. These changes are automatically saved every couple of minutes. Hit the floppy icon to save manually now.
- If you're OK with keeping the container running in the background you can close the jupyter browser windows. To resume, just copy/paste the 127.0.0.1 URL again. Or you could just leave the browser windows running.
- If you want to exit (stop) the container, make sure to save your notebook first. Then, open the Docker Desktop App, find the (running) touchterrain_container under Containers. Hit STOP (__but not DELETE!__) to exit the container. This will also terminate the jupyter server, so you can close its browser windows.

#### Resuming (from a "sleeping" container)
- Although you've exited the container, anything you created or changed within the container is still saved despite being stopped. 
- To resume working with the notebook, open the Docker Desktop App and find the exited (stopped) touchterrain_container under Containers on the right. 
- Click on START and then on CLI, this will give you a container terminal. 
- Type in `bash` and hit enter (this will give you a nicer type of shell). Now type `./run` and hit the tab key, this will result in `./run_touchterrain.sh`. 
- Hit Enter and note the URL starting with 127.0.0.1 that comes up. Copy this URL (with the long token string at the end!) and paste it into a browser. 
- Click on your notebook to resume working with it, it will have all the changes since the last save, including most of the output (except the interactive map and 3D viz).
- To avoid confusion, it's best to clear the output cells and start with a clean slate. Under Kernel, run Restart and Clear Output. Now start running cells from the top (except the authorization cell).

#### Updates 
- At some point, it may be necessary to update either the TouchTerrain module or the entire docker image. For example, there may be new functionality that you want to use. In both cases, your work will be lost or overwritten, so it's imperative to save any files you want to carry over to the updated system first (i.e., download them). This may include your notebook file or any models (zip files) you have created, via the Download button in jupyter.

#### Help, I deleted my container!
- If you removed your container via the Docker Desktop app, either accidentally or b/c you downloaded (pulled) a new docker image, you need to use a OS terminal (PowerShell on Win10 or Terminal.app on Mac) and start the container again using `docker run`. 
- After running the container, you will again have to run the install script (`./install_touchterrain.sh`) and then run jupyter with `./run_touchterrain.sh` and copy/paste the `127.0.0.1` URL into your browser.
- For more see https://github.com/ChHarding/TouchTerrain_jupyter_docker#download-the-image-and-run-the-container
- Once jupyter is running, re-upload anything you need (notebook, data) to the container and resume working